<a href="https://colab.research.google.com/github/philorfa/Biodata/blob/main/WorldClassificationcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title
from __future__ import absolute_import, division, print_function, unicode_literals # legacy compatibility

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util
#@title
!pip install --upgrade pyswarm
!pip install pymc3
!pip install --upgrade pactools

from sklearn.model_selection import train_test_split
from pyswarm import pso
from os import path
import os
import requests
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import numpy
import sys
import numpy as np
from numpy import loadtxt
from numpy import array
from numpy.random import choice
import pandas as pd
import time
import random
import statistics
import pandas
import math
import csv
import random
import logging
from pymc3 import *
import pymc3 as pm
from functools import reduce
from operator import add
from tqdm import tqdm
import geopy.distance
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from theano import shared
from sklearn import preprocessing
print('Running on PyMC3 v{}'.format(pm.__version__))
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow
from tensorflow.keras import datasets, layers, models
from keras.utils import np_utils

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import MaxNLocator

#TESNORFOW
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models
#KERAS LIBRARIES
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout , Flatten,BatchNormalization,Conv2D,MaxPooling2D, Activation,LSTM,Embedding,Input,GlobalAveragePooling2D
from keras.regularizers import l1, l2, l1_l2
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras import backend 
from keras.utils import np_utils

from numpy import savetxt


Requirement already up-to-date: pyswarm in /usr/local/lib/python3.7/dist-packages (0.6)
Requirement already up-to-date: pactools in /usr/local/lib/python3.7/dist-packages (0.3.1)
Running on PyMC3 v3.11.2


In [2]:
X= pd.read_csv('/content/drive/MyDrive/Biodata/Datas/World/X_train_Wor.csv')
X=X.drop(['Unnamed: 0'],axis=1)
y = pd.read_csv('/content/drive/MyDrive/Biodata/Datas/World/y_train_Wor.csv')
y=y.drop(['Unnamed: 0'],axis=1)

Test_data = pd.read_csv('/content/drive/MyDrive/Biodata/Datas/World/X_test_Wor.csv')
Test_data = Test_data.drop(['Unnamed: 0'],axis=1)

Test_labels = pd.read_csv('/content/drive/MyDrive/Biodata/Datas/World/y_test_Wor.csv')
Test_labels = Test_labels.drop(['Unnamed: 0'],axis=1)


In [3]:
print("Shape X_train:", X.shape)
print("Shape y_train:", y.shape)

print("Shape X_test:", Test_data.shape)
print("Shape y_test:", Test_labels.shape)

Shape X_train: (68368, 64)
Shape y_train: (68368, 1)
Shape X_test: (17093, 64)
Shape y_test: (17093, 1)


In [4]:
for i in range(len(y)):
  if (y['Depression_Score'][i]<=7.0):
    y['Depression_Score'][i]=0
  elif (y['Depression_Score'][i]<=12.0):
    y['Depression_Score'][i]=1
  else:
    y['Depression_Score'][i]=2


for i in range(len(Test_labels)):
  if (Test_labels['Depression_Score'][i]<=7.0):
    Test_labels['Depression_Score'][i]=0
  elif (Test_labels['Depression_Score'][i]<=12.0):
    Test_labels['Depression_Score'][i]=1
  else:
    Test_labels['Depression_Score'][i]=2

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y['Depression_Score'], test_size=0.2, random_state=42)

In [ ]:
'''
X_train.to_csv('/content/drive/MyDrive/Biodata/Datas/World/X_train_Wor.csv')
y_train.to_csv('/content/drive/MyDrive/Biodata/Datas/World/y_train_Wor.csv')

X_test.to_csv('/content/drive/MyDrive/Biodata/Datas/World/X_test_Wor.csv')
y_test.to_csv('/content/drive/MyDrive/Biodata/Datas/World/y_test_Wor.csv')
'''

In [6]:
img_rows, img_cols = 8,8

X_train_cnn=np.array(X_train)
X_test_cnn=np.array(X_test)

X_train_cnn = X_train_cnn.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test_cnn = X_test_cnn.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

X_train_cnn = X_train_cnn.astype('float32')
X_test_cnn = X_test_cnn.astype('float32')

print('x_train shape:', X_train_cnn.shape)
print(X_train_cnn.shape[0], 'train samples')
print(X_test_cnn.shape[0], 'test samples')

x_train shape: (54694, 8, 8, 1)
54694 train samples
13674 test samples


In [15]:
EarlyStopper = EarlyStopping(patience=20, monitor='val_accuracy', mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6,patience=20, min_lr=0.000001)
count=0
max=0
lb=[0,0,0,0,0,0]
ub=[0.2,0.2,0.2,0.2,3,3]

In [16]:
def create_model(x):
  print(x[0],x[1],x[2],x[3],x[4],x[5])
  model = Sequential()
  model.add(Conv2D(64, (3, 3), activation='relu', input_shape = input_shape))
  model.add(Conv2D(128, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(x[0]))
  if (x[4]<=1):
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(x[1]))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(x[2]))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(x[3]))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.2))
  elif (x[4]<=2):
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(x[1]))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(x[2]))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(x[3]))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(0.2))
  else:
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(x[1]))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(x[3]))
    model.add(Dense(4, activation='relu'))
    model.add(Dropout(0.2))


  model.add(keras.layers.Dense(3,activation="softmax"))
  model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer='adam', metrics=["accuracy"])
  
  return model

In [21]:
def apple(x):
  global max
  model = create_model(x)
  if (x[5]<1):
    model.fit(X_train_cnn, y_train, epochs=20, batch_size=3000, verbose=1,validation_data=(X_test_cnn, y_test),callbacks=[reduce_lr])
    loss, acc = model.evaluate(X_test_cnn, y_test, verbose=1)
  elif (x[5]<2):
    model.fit(X_train_cnn, y_train, epochs=20, batch_size=1000, verbose=1,validation_data=(X_test_cnn, y_test),callbacks=[reduce_lr])
    loss, acc = model.evaluate(X_test_cnn, y_test, verbose=1)
  else:
    model.fit(X_train_cnn, y_train, epochs=20, batch_size=500, verbose=1,validation_data=(X_test_cnn, y_test),callbacks=[reduce_lr])
    loss, acc = model.evaluate(X_test_cnn, y_test, verbose=1)
  if acc>0.6:
    global count 
    count = count+1
    model.save(f"/content/drive/MyDrive/Biodata/Datas/World/Models_class/model_cnn-{count}-{round(acc, 3)}-{round(loss, 3)}")
    savetxt(f"/content/drive/MyDrive/Biodata/Datas/World/Models_class/data_cnn-{count}.csv", x, delimiter=',')
  return 1/(1+acc)

In [22]:
xopt, fopt = pso(apple, lb, ub, swarmsize=10, omega=0.5, phip=0.5, phig=1.0, maxiter=30, minstep=1)

0.18088596806361676 0.16549068651560447 0.04259528094039906 0.19126444892504746 0.240612738867514 1.8199406433672092
Epoch 1/20
55/55 [==============================] - 3s 20ms/step - loss: 1.0790 - accuracy: 0.3712 - val_loss: 0.8781 - val_accuracy: 0.5636
Epoch 2/20
55/55 [==============================] - 1s 9ms/step - loss: 0.9221 - accuracy: 0.5328 - val_loss: 0.8361 - val_accuracy: 0.5895
Epoch 3/20
55/55 [==============================] - 1s 9ms/step - loss: 0.8893 - accuracy: 0.5609 - val_loss: 0.8258 - val_accuracy: 0.6051
Epoch 4/20
55/55 [==============================] - 0s 9ms/step - loss: 0.8815 - accuracy: 0.5655 - val_loss: 0.8354 - val_accuracy: 0.6058
Epoch 5/20
55/55 [==============================] - 1s 9ms/step - loss: 0.8659 - accuracy: 0.5843 - val_loss: 0.8205 - val_accuracy: 0.6088
Epoch 6/20
55/55 [==============================] - 1s 9ms/step - loss: 0.8496 - accuracy: 0.5955 - val_loss: 0.8179 - val_accuracy: 0.6167
Epoch 7/20
55/55 [========================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Biodata/Datas/World/Models_class/model_cnn-4-0.613-0.816/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Biodata/Datas/World/Models_class/model_cnn-4-0.613-0.816/assets


0.11409548225829175 0.1820549899502547 0.02750546237476195 0.058222222689102335 1.8881276830668727 0.8084992054232222
Epoch 1/20
19/19 [==============================] - 2s 47ms/step - loss: 1.1187 - accuracy: 0.3421 - val_loss: 1.0915 - val_accuracy: 0.3269
Epoch 2/20
19/19 [==============================] - 0s 20ms/step - loss: 1.0738 - accuracy: 0.3953 - val_loss: 0.9143 - val_accuracy: 0.5714
Epoch 3/20
19/19 [==============================] - 0s 20ms/step - loss: 0.9370 - accuracy: 0.5400 - val_loss: 0.8549 - val_accuracy: 0.6043
Epoch 4/20
19/19 [==============================] - 0s 22ms/step - loss: 0.9002 - accuracy: 0.5579 - val_loss: 0.8502 - val_accuracy: 0.6049
Epoch 5/20
19/19 [==============================] - 0s 21ms/step - loss: 0.8908 - accuracy: 0.5628 - val_loss: 0.8403 - val_accuracy: 0.6140
Epoch 6/20
19/19 [==============================] - 0s 21ms/step - loss: 0.8809 - accuracy: 0.5688 - val_loss: 0.8337 - val_accuracy: 0.6132
Epoch 7/20
19/19 [==================

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Biodata/Datas/World/Models_class/model_cnn-5-0.615-0.804/assets


...

0.13344818733396807 0.04237718378657809 0.16633278040732394 0.04681737042051788 1.6650608970041696 2.7572093076918742
Epoch 1/20
110/110 [==============================] - 2s 11ms/step - loss: 1.0384 - accuracy: 0.4146 - val_loss: 0.8448 - val_accuracy: 0.5755
Epoch 2/20
110/110 [==============================] - 1s 7ms/step - loss: 0.8646 - accuracy: 0.5608 - val_loss: 0.8227 - val_accuracy: 0.6075
Epoch 3/20
110/110 [==============================] - 1s 6ms/step - loss: 0.8561 - accuracy: 0.5824 - val_loss: 0.8286 - val_accuracy: 0.6133
Epoch 4/20
110/110 [==============================] - 1s 6ms/step - loss: 0.8515 - accuracy: 0.5899 - val_loss: 0.8129 - val_accuracy: 0.6033
Epoch 5/20
110/110 [==============================] - 1s 6ms/step - loss: 0.8325 - accuracy: 0.6023 - val_loss: 0.8359 - val_accuracy: 0.5929
Epoch 6/20
110/110 [==============================] - 1s 7ms/step - loss: 0.8365 - accuracy: 0.5999 - val_loss: 0.8147 - val_accuracy: 0.6166
Epoch 7/20
110/110 [=========

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Biodata/Datas/World/Models_class/model_cnn-6-0.61-0.819/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Biodata/Datas/World/Models_class/model_cnn-6-0.61-0.819/assets


0.04277529332087842 0.15096996880541053 0.1476316555730157 0.008378074416930437 0.3133093790749798 1.584739117872444
Epoch 1/20


AssertionError: ignored